In [1]:
import pandas as pd
import pyodbc
import logging
import json # For potentially better string representation of results

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

SERVER_NAME = 'localhost'  # Hoặc tên máy chủ/IP của bạn
DB_NAME = 'text_to_sql_final'    # Tên cơ sở dữ liệu của bạn
USER_NAME = 'sa'           # Tên người dùng của bạn
PASSWORD_VAL = '123456'    # Mật khẩu của bạn

CSV_FILE_PATH = 'Golden_v2.csv' # Đường dẫn đến tệp question.csv của bạn
SQL_QUERY_COLUMN_NAME = 'SQL'

print("Các thư viện đã được import và cấu hình đã được thiết lập.")

Các thư viện đã được import và cấu hình đã được thiết lập.


In [2]:
# Ô 3: Hàm Kết nối Cơ sở dữ liệu

def get_db_connection():
    """Thiết lập kết nối đến cơ sở dữ liệu SQL Server."""
    conn_str = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};" # Hoặc trình điều khiển ODBC phù hợp
        f"SERVER={SERVER_NAME};"
        f"DATABASE={DB_NAME};"
        f"UID={USER_NAME};"
        f"PWD={PASSWORD_VAL};"
        f"TrustServerCertificate=yes;" # Quan trọng cho kết nối cục bộ/dev
    )
    try:
        conn = pyodbc.connect(conn_str)
        logging.info(f"Đã kết nối thành công đến SQL Server: {SERVER_NAME}, Cơ sở dữ liệu: {DB_NAME}")
        return conn
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        logging.error(f"Lỗi kết nối cơ sở dữ liệu: {sqlstate} - {ex}")
        if '08001' in sqlstate: # Client unable to establish connection
            logging.error("Lỗi này có thể do: SQL Server chưa chạy, sai tên máy chủ/IP, tường lửa chặn kết nối, hoặc TCP/IP chưa được kích hoạt trên SQL Server.")
            print(f"Không thể kết nối đến máy chủ '{SERVER_NAME}'. Vui lòng kiểm tra lại thông tin và đảm bảo SQL Server đang chạy.")
        elif '28000' in sqlstate: # Invalid authorization specification
            logging.error("Tên người dùng hoặc mật khẩu không hợp lệ.")
            print("Tên đăng nhập hoặc mật khẩu không chính xác.")
        elif '42000' in sqlstate and "Cannot open database" in str(ex):
             logging.error(f"Không thể mở cơ sở dữ liệu '{DB_NAME}'. Kiểm tra lại tên cơ sở dữ liệu.")
             print(f"Không thể mở cơ sở dữ liệu '{DB_NAME}'. Vui lòng kiểm tra lại tên.")
        elif 'IM002' in sqlstate or '01000' in sqlstate: # Data source name not found and no default driver specified
            logging.error("Không tìm thấy ODBC Driver. Hãy đảm bảo 'ODBC Driver 17 for SQL Server' (hoặc trình điều khiển cụ thể của bạn) đã được cài đặt.")
            print("Lỗi ODBC Driver: Không tìm thấy trình điều khiển. Vui lòng cài đặt trình điều khiển ODBC cho SQL Server.")
        else:
            print(f"Lỗi kết nối cơ sở dữ liệu: {ex}")
        return None

print("Hàm get_db_connection() đã được định nghĩa.")

Hàm get_db_connection() đã được định nghĩa.


In [3]:
# Ô 4: Hàm Thực thi Câu lệnh SQL

def execute_sql_query(query_str, conn):
    """Thực thi một câu lệnh SQL và trả về kết quả hoặc thông báo lỗi."""
    if not conn:
        return "Lỗi: Không có kết nối cơ sở dữ liệu."

    cursor = None
    try:
        cursor = conn.cursor()
        logging.info(f"Đang thực thi câu lệnh: {query_str[:200]}...")

        is_select_like_query = query_str.strip().upper().startswith(('SELECT', 'WITH', 'EXEC', 'EXECUTE', 'SHOW'))

        cursor.execute(query_str)

        if is_select_like_query and cursor.description:
            try:
                rows = cursor.fetchall()
                if rows:
                    column_names = [column[0] for column in cursor.description]
                    result_data = [dict(zip(column_names, row)) for row in rows]
                    
                    # --- THAY ĐỔI ---
                    # Thêm `indent=4` để định dạng JSON đầu ra cho dễ đọc.
                    result_str = json.dumps(result_data, ensure_ascii=False, default=str, indent=4)
                else:
                    result_str = "[]"
                logging.info(f"Câu lệnh trả về {len(rows) if rows else 0} dòng.")
            except pyodbc.ProgrammingError as pe:
                logging.warning(f"Lỗi ProgrammingError khi lấy kết quả cho câu lệnh '{query_str[:50]}...': {pe}.")
                if hasattr(cursor, 'rowcount') and cursor.rowcount != -1:
                    result_str = f"{cursor.rowcount} dòng bị ảnh hưởng."
                else:
                    result_str = "Câu lệnh đã thực thi, không có tập kết quả."
            conn.commit()
            return result_str
        else:
            rowcount = cursor.rowcount
            conn.commit()
            logging.info(f"Câu lệnh đã thực thi. Số dòng bị ảnh hưởng: {rowcount}")
            return f"{rowcount} dòng bị ảnh hưởng."

    except pyodbc.Error as e:
        error_message = f"Lỗi SQL: {e}"
        logging.error(error_message + f" cho câu lệnh: {query_str}")
        try:
            conn.rollback()
        except pyodbc.Error as r_err:
            logging.error(f"Rollback thất bại: {r_err}")
        return error_message
    finally:
        if cursor:
            cursor.close()

print("Hàm execute_sql_query() đã được định nghĩa với định dạng JSON cải tiến.")

Hàm execute_sql_query() đã được định nghĩa với định dạng JSON cải tiến.


In [4]:
# Ô 5: Hàm Logic Xử lý Chính

def process_queries_and_update_csv():
    """
    Đọc tệp CSV, thực thi các câu lệnh SQL, và cập nhật tệp CSV với kết quả.
    """
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        logging.info(f"Đã đọc thành công tệp {CSV_FILE_PATH}. Tìm thấy {len(df)} câu lệnh.")
    except FileNotFoundError:
        logging.error(f"Lỗi: Không tìm thấy tệp {CSV_FILE_PATH}.")
        print(f"Lỗi: Không tìm thấy tệp {CSV_FILE_PATH}. Vui lòng đảm bảo tệp nằm trong thư mục chính xác.")
        return
    except Exception as e:
        logging.error(f"Lỗi khi đọc tệp CSV {CSV_FILE_PATH}: {e}")
        print(f"Lỗi khi đọc tệp CSV {CSV_FILE_PATH}: {e}")
        return

    if SQL_QUERY_COLUMN_NAME not in df.columns:
        logging.error(f"Lỗi: Không tìm thấy cột '{SQL_QUERY_COLUMN_NAME}' trong {CSV_FILE_PATH}.")
        print(f"Lỗi: Không tìm thấy cột '{SQL_QUERY_COLUMN_NAME}' trong {CSV_FILE_PATH}. Vui lòng kiểm tra lại tên cột.")
        return

    if df.empty:
        logging.info(f"Tệp {CSV_FILE_PATH} trống. Không có câu lệnh nào để xử lý.")
        print(f"Tệp {CSV_FILE_PATH} trống. Không có câu lệnh nào để xử lý.")
        df['Result'] = []
        try:
            df.to_csv(CSV_FILE_PATH, index=False, encoding='utf-8-sig')
            logging.info(f"Đã thêm cột 'Result' vào tệp {CSV_FILE_PATH} trống.")
        except Exception as e:
            logging.error(f"Lỗi khi ghi vào tệp CSV {CSV_FILE_PATH}: {e}")
            print(f"Lỗi khi ghi vào tệp CSV {CSV_FILE_PATH}: {e}")
        return

    conn = get_db_connection()
    if not conn:
        logging.warning("Không thể thiết lập kết nối cơ sở dữ liệu. Các câu lệnh sẽ không được thực thi.")
        print("Không thể kết nối đến cơ sở dữ liệu. Vui lòng kiểm tra thông tin kết nối và cài đặt ODBC driver.")
        df['Result'] = "Lỗi: Kết nối cơ sở dữ liệu thất bại."
        try:
            df.to_csv(CSV_FILE_PATH, index=False, encoding='utf-8-sig')
            logging.info(f"Đã cập nhật {CSV_FILE_PATH} với trạng thái lỗi kết nối.")
            print(f"Đã cập nhật {CSV_FILE_PATH} với trạng thái lỗi kết nối cho tất cả các câu lệnh.")
        except Exception as e:
            logging.error(f"Lỗi khi ghi vào tệp CSV {CSV_FILE_PATH}: {e}")
            print(f"Lỗi khi ghi vào tệp CSV {CSV_FILE_PATH}: {e}")
        return

    results = []
    print(f"Bắt đầu xử lý {len(df)} câu lệnh SQL...")
    for index, row in df.iterrows():
        query = row[SQL_QUERY_COLUMN_NAME]
        if pd.isna(query) or not str(query).strip():
            logging.warning(f"Bỏ qua câu lệnh rỗng hoặc NaN ở dòng {index}.")
            results.append("Bỏ qua: Câu lệnh rỗng")
            continue
        
        print(f"Đang thực thi câu lệnh {index + 1}/{len(df)}: {str(query)[:100]}...")
        query_result = execute_sql_query(str(query), conn)
        results.append(query_result)

    if conn:
        conn.close()
        logging.info("Đã đóng kết nối cơ sở dữ liệu.")

    df['Result'] = results

    try:
        df.to_csv(CSV_FILE_PATH, index=False, encoding='utf-8-sig')
        logging.info(f"Đã cập nhật thành công tệp {CSV_FILE_PATH} với kết quả câu lệnh.")
        print(f"\nXử lý hoàn tất! Tệp '{CSV_FILE_PATH}' đã được cập nhật với cột 'result'.")
    except Exception as e:
        logging.error(f"Lỗi khi ghi DataFrame đã cập nhật vào CSV: {e}")
        print(f"Lỗi khi ghi kết quả vào tệp {CSV_FILE_PATH}: {e}")

print("Hàm process_queries_and_update_csv() đã được định nghĩa.")

Hàm process_queries_and_update_csv() đã được định nghĩa.


In [5]:
# Ô 6: Gọi hàm để thực thi

print("Bắt đầu quá trình xử lý câu lệnh SQL từ tệp CSV...")
process_queries_and_update_csv()
print("Kịch bản đã hoàn thành.")

2025-06-12 17:42:12,853 - INFO - Đã đọc thành công tệp Golden_v2.csv. Tìm thấy 86 câu lệnh.


Bắt đầu quá trình xử lý câu lệnh SQL từ tệp CSV...


2025-06-12 17:42:12,988 - INFO - Đã kết nối thành công đến SQL Server: localhost, Cơ sở dữ liệu: text_to_sql_final
2025-06-12 17:42:12,989 - INFO - Đang thực thi câu lệnh: SELECT
  COUNT(T1.status)
FROM Attendance AS T1
INNER JOIN Enrollments AS T2
  ON T1.enrollment_id = T2.enrollment_id
INNER JOIN ClassCourse AS T3
  ON T2.class_course_id = T3.class_course_id
INNER JO...
2025-06-12 17:42:13,001 - INFO - Câu lệnh trả về 1 dòng.


Bắt đầu xử lý 86 câu lệnh SQL...
Đang thực thi câu lệnh 1/86: SELECT
  COUNT(T1.status)
FROM Attendance AS T1
INNER JOIN Enrollments AS T2
  ON T1.enrollment_id =...


2025-06-12 17:42:13,004 - INFO - Đang thực thi câu lệnh: SELECT
  COUNT(T1.status)
FROM Attendance AS T1
JOIN Enrollments AS T2
  ON T1.enrollment_id = T2.enrollment_id
JOIN ClassCourse AS T3
  ON T2.class_course_id = T3.class_course_id
JOIN Courses AS T4
 ...
2025-06-12 17:42:13,012 - INFO - Câu lệnh trả về 1 dòng.
2025-06-12 17:42:13,014 - INFO - Đang thực thi câu lệnh: SELECT
    COUNT(DISTINCT T1.student_id)
FROM Enrollments AS T1
JOIN ClassCourse AS T2
    ON T1.class_course_id = T2.class_course_id
WHERE
    T2.semester = 'SU24'...
2025-06-12 17:42:13,021 - INFO - Câu lệnh trả về 1 dòng.
2025-06-12 17:42:13,023 - INFO - Đang thực thi câu lệnh: SELECT
    C.course_id,
    C.course_name,
    COUNT(CASE WHEN E.status = 'Passed' THEN 1 END) AS passed_count,
    COUNT(CASE WHEN E.status = 'Failed' THEN 1 END) AS failed_count
FROM Enrollments AS ...
2025-06-12 17:42:13,035 - INFO - Câu lệnh trả về 4 dòng.


...

Đang thực thi câu lệnh 81/86: SELECT
  C.course_name,
  SUM(CASE WHEN E.status = 'Passed' THEN 1 ELSE 0 END) AS pass_count,
  SUM(...


2025-06-12 17:42:13,752 - INFO - Đang thực thi câu lệnh: SELECT
    C.course_name,
    SUM(CASE WHEN E.status = 'Passed' THEN 1 ELSE 0 END) AS passed_count,
    SUM(CASE WHEN E.status = 'Failed' THEN 1 ELSE 0 END) AS failed_count
FROM Enrollments AS E
JOIN ...
2025-06-12 17:42:13,763 - INFO - Câu lệnh trả về 4 dòng.
2025-06-12 17:42:13,764 - INFO - Đang thực thi câu lệnh: SELECT TOP 3 WITH TIES
    C.course_name,
    CAST(SUM(CASE WHEN E.status = 'Failed' THEN 1 ELSE 0 END) AS FLOAT) 
        * 100.0 / COUNT(E.enrollment_id) AS failure_rate
FROM 
    Enrollments AS E
J...
2025-06-12 17:42:13,774 - INFO - Câu lệnh trả về 3 dòng.
2025-06-12 17:42:13,776 - INFO - Đang thực thi câu lệnh: SELECT
  U.fullname,
  M.major_name,
  COUNT(E.enrollment_id) AS number_of_failed_courses
FROM Enrollments AS E
JOIN Students AS S
  ON E.student_id = S.student_id
JOIN Users AS U
  ON S.student_id = ...
2025-06-12 17:42:13,787 - INFO - Câu lệnh trả về 2 dòng.


Đang thực thi câu lệnh 82/86: SELECT
    C.course_name,
    SUM(CASE WHEN E.status = 'Passed' THEN 1 ELSE 0 END) AS passed_count,
...
Đang thực thi câu lệnh 83/86: SELECT TOP 3 WITH TIES
    C.course_name,
    CAST(SUM(CASE WHEN E.status = 'Failed' THEN 1 ELSE 0 E...
Đang thực thi câu lệnh 84/86: SELECT
  U.fullname,
  M.major_name,
  COUNT(E.enrollment_id) AS number_of_failed_courses
FROM Enrol...


2025-06-12 17:42:13,789 - INFO - Đang thực thi câu lệnh: SELECT
    T3.grade_name,
    T1.grade_value,
    T1.comment
FROM StudentGradeDetails AS T1
INNER JOIN Enrollments AS T2
    ON T1.enrollment_id = T2.enrollment_id
...ER JOIN CourseGrade AS T3
2025-06-12 17:42:13,798 - INFO - Câu lệnh trả về 7 dòng.
2025-06-12 17:42:13,799 - INFO - Đang thực thi câu lệnh: SELECT
  U.fullname,
  COUNT(CC.course_id) AS number_of_courses
FROM Users AS U
JOIN Lecturers AS L
  ON U.user_id = L.lecturer_id
JOIN ClassCourse AS CC
  ON L.lecturer_id = CC.lecturer_id
WHERE
  CC...
2025-06-12 17:42:13,806 - INFO - Câu lệnh trả về 6 dòng.
2025-06-12 17:42:13,807 - INFO - Đã đóng kết nối cơ sở dữ liệu.
2025-06-12 17:42:13,816 - INFO - Đã cập nhật thành công tệp Golden_v2.csv với kết quả câu lệnh.


Đang thực thi câu lệnh 85/86: SELECT
    T3.grade_name,
    T1.grade_value,
    T1.comment
FROM StudentGradeDetails AS T1
INN...
Đang thực thi câu lệnh 86/86: SELECT
  U.fullname,
  COUNT(CC.course_id) AS number_of_courses
FROM Users AS U
JOIN Lecturers AS L
...

Xử lý hoàn tất! Tệp 'Golden_v2.csv' đã được cập nhật với cột 'result'.
Kịch bản đã hoàn thành.
